### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"
# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
#Setup Formatting
def dec_fmt(d_in, is_cash=False, is_perc=False, use_commas=False):
    fmtstr = ''
    if is_cash:
        fmtstr += '$'
    fmtstr += '{:'
    if use_commas:
        fmtstr += ','
    fmtstr += '.2f}'
    if is_perc:
        fmtstr += '%'
    return d_in.map(fmtstr.format)

from functools import partial
perc_fmt = partial(dec_fmt, is_perc=True)
cash_fmt = partial(dec_fmt, is_cash=True)
cashcomma_fmt = partial(dec_fmt, is_cash=True, use_commas=True)

## Player Count

* Display the total number of players


In [3]:
# Find all unique player screen names.
unique_players = purchase_data["SN"].nunique()
# Get the total number of players.
print(unique_players)

576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
# Create variables for the summary table.
unique_items = purchase_data["Item ID"].nunique()
price_average = purchase_data["Price"].mean()
total_purchases = len(purchase_data)
total_revenue = purchase_data["Price"].sum()
# Create a list of lists using the new variables.
ST = [[unique_items, price_average, total_purchases, total_revenue]] 
columns = ["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]
# Create the drataframe for our summary table.
purchase_totals = pd.DataFrame(ST, columns=columns)
# Formating for the USD columns.
purchase_totals["Average Price"] = cash_fmt(purchase_totals["Average Price"])
purchase_totals["Total Revenue"] = cashcomma_fmt(purchase_totals["Total Revenue"])
# Get that data!
print(purchase_totals.to_string(index=False))

 Number of Unique Items Average Price  Number of Purchases Total Revenue
                    179         $3.05                  780     $2,379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
# Group purchase_data by Gender
gender_groups = purchase_data.groupby(["Gender"])
# Create df with the sum of the unique screen names
gender_count = pd.DataFrame(gender_groups["SN"].nunique())
# Create new df with percentage values
gender_percentage = (gender_count/unique_players)*100
# Merge the two dataframes by Gender
gender_summary = pd.merge(gender_count, gender_percentage, on="Gender", how="right")
# Cleanup and format the dataframe
gender_summary.columns = ["Total Count", "Percentage of Players"]
gender_summary["Percentage of Players"] = perc_fmt(gender_summary["Percentage of Players"])
gender_summary.sort_values(by=["Total Count"], inplace=True, ascending=False)
gender_summary.index.name = None
# Get that data!
print(gender_summary.to_string(index=True))

                       Total Count Percentage of Players
Male                           484                84.03%
Female                          81                14.06%
Other / Non-Disclosed           11                 1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
# Run calculations for dataframe
purchase_count = pd.DataFrame(gender_groups["Purchase ID"].count())
average_purchase = pd.DataFrame(gender_groups["Price"].mean())
total_purchased = pd.DataFrame(gender_groups["Price"].sum())
average_person = total_purchased.div(gender_count.values)
# Merge everything together to create new df
purchase_list = [purchase_count, average_purchase, total_purchased, average_person]
merged_purchases = pd.concat(purchase_list, join="outer", axis=1)
# Formatting stuff
merged_purchases.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]
merged_purchases["Average Purchase Price"] = cash_fmt(merged_purchases["Average Purchase Price"])
merged_purchases["Total Purchase Value"] = cashcomma_fmt(merged_purchases["Total Purchase Value"])
merged_purchases["Avg Total Purchase per Person"] = cash_fmt(merged_purchases["Avg Total Purchase per Person"])
# Get that data!
print(merged_purchases.to_string(index=True))

                       Purchase Count Average Purchase Price Total Purchase Value Avg Total Purchase per Person
Gender                                                                                                         
Female                            113                  $3.20              $361.94                         $4.47
Male                              652                  $3.02            $1,967.64                         $4.07
Other / Non-Disclosed              15                  $3.35               $50.19                         $4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
# Make a bin
age_bin = [0, 9, 14, 19, 24, 29, 34, 39, 200]
bin_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
# Add age values to the bin
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"], age_bin, labels=bin_names)
# Dataframe for the new column
age_groups = purchase_data.groupby("Age Ranges")
# Calculate Totals & Percentage
age_totals = pd.DataFrame(age_groups["SN"].nunique())
age_percentage = (age_totals/unique_players)*100
# Merge dataframes
age_summary = pd.merge(age_totals, age_percentage, on="Age Ranges", how="outer")
# Formatting new dataframe
age_summary.columns = ["Total Count", "Percentage of Players"]
age_summary["Percentage of Players"] = perc_fmt(age_summary["Percentage of Players"])
age_summary.index.name = None
# Get that data!
print(age_summary.to_string(index=True))

       Total Count Percentage of Players
<10             17                 2.95%
10-14           22                 3.82%
15-19          107                18.58%
20-24          258                44.79%
25-29           77                13.37%
30-34           52                 9.03%
35-39           31                 5.38%
40+             12                 2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
# Run calculations for dataframe
age_purchase_count = pd.DataFrame(age_groups["Purchase ID"].count())
age_average_purchase = pd.DataFrame(age_groups["Price"].mean())
age_total_purchased = pd.DataFrame(age_groups["Price"].sum())
age_average_person = age_total_purchased.div(age_totals.values)
# Merge everything together to create new df
age_purchase_list = [age_purchase_count, age_average_purchase, age_total_purchased, age_average_person]
age_merged = pd.concat(age_purchase_list, join="outer", axis=1)
# Formatting stuff
age_merged.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]
age_merged["Average Purchase Price"] = cash_fmt(age_merged["Average Purchase Price"])
age_merged["Total Purchase Value"] = cashcomma_fmt(age_merged["Total Purchase Value"])
age_merged["Avg Total Purchase per Person"] = cash_fmt(age_merged["Avg Total Purchase per Person"])
# Get that data!
print(age_merged.to_string(index=True))

            Purchase Count Average Purchase Price Total Purchase Value Avg Total Purchase per Person
Age Ranges                                                                                          
<10                     23                  $3.35               $77.13                         $4.54
10-14                   28                  $2.96               $82.78                         $3.76
15-19                  136                  $3.04              $412.89                         $3.86
20-24                  365                  $3.05            $1,114.06                         $4.32
25-29                  101                  $2.90              $293.00                         $3.81
30-34                   73                  $2.93              $214.00                         $4.12
35-39                   41                  $3.60              $147.67                         $4.76
40+                     13                  $2.94               $38.24                     

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [12]:
# Group data by screen names
spender_group = purchase_data.groupby("SN")
# Unique number of spenders for later
unique_spenders = pd.DataFrame(spender_group["SN"].nunique())
# Run all the calculations
spender_count = spender_group["Purchase ID"].count()
spender_average = pd.DataFrame(spender_group["Price"].mean())
spender_total = pd.DataFrame(spender_group["Price"].sum())
# Merge everything together to create new df
spender_list = [spender_count, spender_average, spender_total]
spenders_merged = pd.concat(spender_list, join="outer", axis=1)
# Formating dataframe
spenders_merged.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]
spenders_merged["Average Purchase Price"] = cash_fmt(spenders_merged["Average Purchase Price"])
spenders_merged["Total Purchase Value"] = cashcomma_fmt(spenders_merged["Total Purchase Value"])
spenders_merged = spenders_merged.sort_values(by=["Purchase Count"], ascending=False)
#Get that data!
print(spenders_merged.to_string(index=True, max_rows=6))

              Purchase Count Average Purchase Price Total Purchase Value
SN                                                                      
Lisosia93                  5                  $3.79               $18.96
Iral74                     4                  $3.40               $13.62
Idastidru52                4                  $3.86               $15.45
Asur53                     3                  $2.48                $7.44
Inguron55                  3                  $3.70               $11.11
Aina42                     3                  $3.07                $9.22
Ilarin91                   3                  $4.23               $12.70
Idai61                     3                  $2.74                $8.23
Umolrian85                 3                  $2.71                $8.13
Ialallo29                  3                  $3.95               $11.84
...                      ...                    ...                  ...
Heudai45                   1                  $3.47

## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
# Create df with the item columns
item_data = purchase_data[["Item ID", "Item Name", "Price"]]
# Group items based on ID and name
item_group = item_data.groupby(["Item ID", "Item Name"])
# Run calcuations for new table
purchased_items = pd.DataFrame(item_group["Price"].count())
item_totals = pd.DataFrame(item_group["Price"].sum())
item_prices = item_totals.div(purchased_items.values)
# Merge it all
items_list = [purchased_items, item_prices, item_totals]
items_merged = pd.concat(items_list, join="outer", axis=1)
# Format everything
items_merged.columns = ["Purchase Count", "Item Price", "Total Purchase Value"]
items_merged["Item Price"] = cash_fmt(items_merged["Item Price"])
items_merged["Total Purchase Value"] = cashcomma_fmt(items_merged["Total Purchase Value"])
items_merged = items_merged.sort_values(by=["Purchase Count"], ascending=False)
# Get that data!
print(items_merged.to_string(index=True, max_rows=6))

                                                      Purchase Count Item Price Total Purchase Value
Item ID Item Name                                                                                   
92      Final Critic                                              13      $4.61               $59.99
178     Oathbreaker, Last Hope of the Breaking Storm              12      $4.23               $50.76
145     Fiery Glass Crusader                                       9      $4.58               $41.22
...                                                              ...        ...                  ...
118     Ghost Reaver, Longsword of Magic                           1      $2.17                $2.17
104     Gladiator's Glaive                                         1      $1.93                $1.93
91      Celeste                                                    1      $4.17                $4.17


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [11]:
# Copy dataframe
items_profitable = items_merged
# Remove symbol and convert column into float type for sorting
items_profitable["Total Purchase Value"] = items_profitable["Total Purchase Value"].str.replace("$", "").astype("float")
# Sort data
items_profitable = items_profitable.sort_values(by=["Total Purchase Value"], ascending=False)
# Re-Format
items_profitable["Total Purchase Value"] = cashcomma_fmt(items_profitable["Total Purchase Value"])
# Get that data!
print(items_profitable.to_string(index=True, max_rows=6))

                                                      Purchase Count Item Price Total Purchase Value
Item ID Item Name                                                                                   
92      Final Critic                                              13      $4.61               $59.99
178     Oathbreaker, Last Hope of the Breaking Storm              12      $4.23               $50.76
82      Nirvana                                                    9      $4.90               $44.10
...                                                              ...        ...                  ...
125     Whistling Mithril Warblade                                 2      $1.00                $2.00
104     Gladiator's Glaive                                         1      $1.93                $1.93
42      The Decapitator                                            1      $1.75                $1.75
